### Testing execution time

In [ ]:
from project_sparse import *
from scipy import sparse
import random

def randmatrix(approximate_sparsity, num_rows, num_cols):
    """
    Randomly generates a matrix - for testing purposes.

    Parameters
    ----------
    approximate_sparsity : integer
        The approximate ratio of zeros in relation to other values.
    num_rows : integer
        The number of rows in the randomly generated matrix.
    num_cols : integer
        The number of columns in the randomly generated matrix.

    Returns
    -------
    matrix : np.array
        The randomly generated matrix.

    """
    matrix = np.zeros(shape=[num_rows, num_cols])
    for i in range(num_rows):
        for j in range(num_cols):
            num = random.random()
            if num > approximate_sparsity:
                matrix[i,j] = (random.random()-0.5)*1000
    return matrix

n = 1000

matrices = [randmatrix(random.random(),
random.randint(3,30),
random.randint(3,30)) for i in range(n)]

matrices2 = [randmatrix(random.random(),
matrices[i].shape[0],
matrices[i].shape[1]) for i in range(n)]

vectors = []
for i in range(n):
    vectors.append(*randmatrix(random.random(),
    1,
    matrices[i].shape[1]))

our = [SparseMatrix(matrices[i], tol=0) for i in range(n)]
sci = [sparse.csr_matrix(matrices[i]) for i in range(n)]

our2 = [SparseMatrix(matrices2[i], tol=0) for i in range(n)]
sci2 = [sparse.csr_matrix(matrices2[i]) for i in range(n)]

ourc = [SparseMatrix(matrices2[i], tol=0) for i in range(n)]
for i in range(n):
    ourc[i].switch('CSC')
scic = [sparse.csc_matrix(matrices2[i]) for i in range(n)]

Changing an element present in a CSR matrix

###### A warning on efficiency is thrown by scipy here. It is ignored since our class doesn't have a lil_matrix type that would allow for a direct comparison.



In [ ]:
%timeit [our[i].change(1,2,25) for i in range(n)]
%timeit [sci[i].__setitem__((1,2),25) for i in range(n)]

Summing up two CSR matrices

In [ ]:
%timeit [our[i].add(our2[i]) for i in range(n)]
%timeit [sci[i] + sci2[i] for i in range(n)]

Summing up a CSR matrix and a CSC matrix

In [ ]:
%timeit [our[i].add(ourc[i]) for i in range(n)]
%timeit [sci[i] + scic[i] for i in range(n)]

Multiplying a CSR matrix with a vector

In [ ]:
%timeit [our[i].multiply(vectors[i]) for i in range(n)]
%timeit [sci[i] * vectors[i] for i in range(n)]

### Comparing outputs

In [ ]:
def toarray(SparseMatrix):
    """
    Converts a sparse matrix to type np.array - for testing purposes.

    Parameters
    ----------
    SparseMatrix : SparseMatrix
        A sparse matrix to be converted.

    Returns
    -------
    result : np.array
        The sparse matrix in np.array format.

    """
    Sum = copy.copy(SparseMatrix)
    if Sum.intern_represent == 'CSR':
        Sum.rows = decompress(Sum.rows, Sum.num_rows)
    else:
        Sum.switch('CSR')
        Sum.rows = decompress(Sum.rows, Sum.num_rows)

    result = np.zeros((Sum.num_rows, Sum.num_cols))
    for i in range(Sum.number_of_nonzero):
        result[Sum.rows[i], Sum.cols[i]]=Sum.v[i]
    return result

def compare():
    """
    Compares whether ourf and scif are the same and prints them if they differ - for testing purposes.

    Parameters
    ----------
    None.

    Returns
    -------
    None.

    """
    for i in range(n):
        if (toarray(ourf[i]) != scif[i].toarray()).any():
            print(scif[i])
            ourf[i].show()

ourf = copy.copy(our)
scif = copy.copy(sci)
compare()
ourf = copy.copy(our2)
scif = copy.copy(sci2)
compare()
ourf = copy.copy(ourc)
scif = copy.copy(scic)
compare()

Changing an element present in a CSR matrix

In [ ]:
for i in range(n):
    ourf[i].change(1,2,25)
    scif[i].__setitem__((1,2),25)

compare()

Summing up two CSR matrices

In [ ]:
for i in range(n):
    ourf[i] = our[i].add(our2[i])
    scif[i] = sci[i] + sci2[i]

compare()

Summing up a CSR matrix and a CSC matrix

In [ ]:
for i in range(n):
    ourf[i] = our[i].add(ourc[i])
    scif[i] = sci[i] + scic[i]

compare()

Summing up two CSC matrices

In [ ]:
for i in range(n):
    ourf[i] = copy.copy(our[i])
    ourf[i].switch('CSC')
    ourf[i] = ourc[i].add(ourf[i])
    scif[i] = scic[i] + sci[i].tocsc()

compare()

Multiplying a matrix with a vector

In [ ]:
# CSR
for i in range(n):
    if (our[i].multiply(vectors[i]) != sci[i] * vectors[i]).any():
        print(our[i].multiply(vectors[i]))
        print(sci[i] * vectors[i])
# CSC
for i in range(n):
    if (ourc[i].multiply(vectors[i]) != scic[i] * vectors[i]).any():
        print(ourc[i].multiply(vectors[i]))
        print(scic[i] * vectors[i])